In [1]:
# load dataset and print out head.
from datasets import load_dataset

jnlpba = load_dataset('jnlpba')

# Print the first example from the dataset
print(jnlpba['train'][0])

Generating train split:   0%|          | 0/37094 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7714 [00:00<?, ? examples/s]

Dataset jnlpba downloaded and prepared to /Users/maxhager/.cache/huggingface/datasets/jnlpba/jnlpba/1.0.0/3062f220823930cffde7976b694aa67bac3b06c322a02ced92d3761519810ce4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'id': '1', 'tokens': ['IL-2', 'gene', 'expression', 'and', 'NF-kappa', 'B', 'activation', 'through', 'CD28', 'requires', 'reactive', 'oxygen', 'production', 'by', '5-lipoxygenase', '.'], 'ner_tags': [1, 2, 0, 0, 9, 10, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0]}


In [4]:
full_names_tags = {"O": "Outside", "B-DNA": "Gene Begin", "I-DNA": "Gene Inside", "B-PROTEIN": "Protein Begin", "I-PROTEIN": "Protein Inside", "B-CELL-LINE": "Cell Line Begin", "I-CELL-LINE": "Cell Line Inside", "B-CELL-TYPE": "Cell Type Begin", "I-CELL-TYPE": "Cell Type Inside", "B-DISEASE": "Disease Begin", "I-DISEASE": "Disease Inside"}

11
